<a href="https://colab.research.google.com/github/advait-t/IPL_Datasets/blob/main/Dream11_Team.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from itertools import permutations 
import pandas as pd
import numpy as np

In [2]:
def playing_x1(data, team, team1, team2):
  data = data.iloc[:,1:]
  data['Team'] = data['Team'].apply(lambda x : x.strip())
  data['Name'] = data['Name'].apply(lambda x : x.strip())
  data['Playing'] = data['Playing'].apply(lambda x : x.strip())
  data['Nationality'] = data['Nationality'].apply(lambda x : x.strip())

  temp = pd.concat([data[data['Team']==team1],data[data['Team']==team2]],axis=0)
  team['Credits'] = 0
  team['Category'] = np.nan
  team['Team'] = np.nan

  for i in range(0, team.shape[0]):
    for j in range(0, temp.shape[0]):
      #print(team.iloc[i,0], temp.iloc[j,1])
      if team.iloc[i,0]==temp.iloc[j,1].strip():
        #print(team.iloc[i,0], temp.iloc[j,4], temp.iloc[j,2])
        team.iloc[i,4] = temp.iloc[j,4]
        team.iloc[i,5] = temp.iloc[j,2]
        team.iloc[i,6] = temp.iloc[j,0]
        if team.iloc[i,5] == "Batsman":
          team.iloc[i,2] = 0
        if team.iloc[i,5] == "Bowler":
          team.iloc[i,1] = 0
        team.iloc[i,3] = team.iloc[i,1] + team.iloc[i,2]

  team['Category'][team['Category']=="Batting Allrounder"] = "Allrounder Batting"
  team['Category'][team['Category']=="Bowling Allrounder"] = "Allrounder Bowling"
  team = team.sort_values(by =['Category', 'Total Points'], ascending=False)
  team = team.dropna()
  return team

In [3]:
def fantasy_team_creator(team, team1, team2):
  WK = team[team['Category']=="WK-Batsman"].shape[0]
  BOL = team[team['Category']=="Bowler"].shape[0]
  BAT = team[team['Category']=="Batsman"].shape[0]
  AR = team.shape[0]-WK-BOL-BAT

  total_player = 11
  team_conf = []

  for j in range(1, min(WK,4)+1):
    for l in range(3, min(BAT,6)+1):
      for n in range(1, min(AR,4)+1):
        for p in range(3, min(BOL,6)+1):
          if j+l+n+p == 11:
            print("Team:", "WK - ", j, "BAT - ", l, "AR - ",  n, "BOL - ",  p)
            team_conf.append([j,l,n,p])

  wk_lt, bat_lt, ar_lt, bol_lt = [],[],[],[]

  for i in range(1,min(WK,4)+1):
    wk_lt.append(list(set(permutations([1] * i + [0] * (WK-i)))))

  for i in range(3,min(BAT,6)+1):
    bat_lt.append(list(set(permutations([1] * i + [0] * (BAT-i)))))

  for i in range(1,min(AR,4)+1):
    ar_lt.append(list(set(permutations([1] * i + [0] * (AR-i)))))

  for i in range(3,min(BOL,6)+1):
    bol_lt.append(list(set(permutations([1] * i + [0] * (BOL-i)))))

  conf = []
  for c in team_conf:

    wk,bt,ar,bl = c[0],c[1],c[2],c[3]
    print(wk,bt,ar,bl)

    for i in wk_lt[wk-1]:
      for j in bat_lt[bt-3]:
        for k in ar_lt[ar-1]:
          for l in bol_lt[bl-3]:
            temp = []
            #print(i,j,k,l)
            for x in i: temp.append(x)
            for x in l: temp.append(x)
            for x in j: temp.append(x)
            for x in k: temp.append(x)
            conf.append(temp)
  
  team["Is In FT"] = 0
  best_fp = 0
  for j in conf:
    total_fp = 0
    if np.dot(team.iloc[:,4],j) <= 100:
      total_fp = total_fp + np.dot(team.iloc[:,3], j)
      if total_fp > best_fp:
        tc1, tc2 = list(team["Team"]*j).count(team1), list(team["Team"]*j).count(team2)
        if tc1>=4 and tc2>=4:
          best_fp = total_fp
          team["Is In FT"] = j
          print(best_fp, np.dot(team.iloc[:,4],j))
        #print(total_fp)
  fantasy_team = team[team['Is In FT']==1]

  return fantasy_team

In [5]:
data = pd.read_csv("/content/Players 2021.csv")
team = pd.read_csv("/content/tempteam.csv")

team = playing_x1(data, team, "Mumbai Indians", "Rajasthan Royals")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [6]:
team

,Player,Batting Points,Bowling Points,Total Points,Credits,Category,Team
0,Sanju Samson,68.0,0.0,68,9.5,WK-Batsman,Rajasthan Royals
1,Jos Buttler,45.0,0.0,45,9.5,WK-Batsman,Rajasthan Royals
15,Quinton de Kock,18.0,0.0,18,9.5,WK-Batsman,Mumbai Indians
21,Jasprit Bumrah,0.0,50.0,50,9.0,Bowler,Mumbai Indians
17,Trent Boult,0.0,25.0,25,9.0,Bowler,Mumbai Indians
8,Jaydev Unadkat,0.0,22.0,22,8.0,Bowler,Rajasthan Royals
18,Rahul Chahar,0.0,3.0,3,8.5,Bowler,Mumbai Indians
9,Chetan Sakariya,0.0,0.0,0,8.0,Bowler,Rajasthan Royals
19,Nathan Coulter-Nile,0.0,0.0,0,8.0,Bowler,Mumbai Indians
20,Jayant Yadav,0.0,0.0,0,8.0,Bowler,Mumbai Indians


In [ ]:
fantasy_team_creator(team, "Mumbai Indians", "Rajasthan Royals")

Team: WK -  1 BAT -  3 AR -  1 BOL -  6
Team: WK -  1 BAT -  3 AR -  2 BOL -  5
Team: WK -  1 BAT -  3 AR -  3 BOL -  4
Team: WK -  1 BAT -  3 AR -  4 BOL -  3
Team: WK -  1 BAT -  4 AR -  1 BOL -  5
Team: WK -  1 BAT -  4 AR -  2 BOL -  4
Team: WK -  1 BAT -  4 AR -  3 BOL -  3
Team: WK -  1 BAT -  5 AR -  1 BOL -  4
Team: WK -  1 BAT -  5 AR -  2 BOL -  3
Team: WK -  2 BAT -  3 AR -  1 BOL -  5
Team: WK -  2 BAT -  3 AR -  2 BOL -  4
Team: WK -  2 BAT -  3 AR -  3 BOL -  3
Team: WK -  2 BAT -  4 AR -  1 BOL -  4
Team: WK -  2 BAT -  4 AR -  2 BOL -  3
Team: WK -  2 BAT -  5 AR -  1 BOL -  3
Team: WK -  3 BAT -  3 AR -  1 BOL -  4
Team: WK -  3 BAT -  3 AR -  2 BOL -  3
Team: WK -  3 BAT -  4 AR -  1 BOL -  3
1 3 1 6
1 3 2 5
1 3 3 4
1 3 4 3
1 4 1 5
1 4 2 4
1 4 3 3
1 5 1 4
1 5 2 3
2 3 1 5
2 3 2 4
2 3 3 3
2 4 1 4
2 4 2 3
2 5 1 3
3 3 1 4
3 3 2 3
3 4 1 3
323.0 96.5
332.0 96.5
333.0 96.5
383.0 96.5
392.0 96.5
425.0 97.5
431.0 97.0
434.0 97.5
437.0 98.0
456.0 97.5
459.0 98.0
467.0 98.0
470.

In [ ]:
team